## Tokenization is at the heart of LLMs

<small> Simple string processing can be difficult due to tokenization for LLM </small>

<small> Sometimes LLMs are bad at artithmetic due to tokenization </small>

<small> Earlier versions of gpt had issues with Python due to tokenization </small>

<small> LLMs do worse for non english languages due to tokenization </small>

<small> Using YAML over json due to tokenization </small>

<small> tiktokenizer.vercel.app is good for visualizing tokenization </small>

<small>So like even for the same concept "egg" can be very different tokens and ids. The model has to somehow learn that these are the same concept and group them in the nn properly. </small>
<small> For GPT4, their tokenizer is able to group white spaces into one token which is great for its Python coding ability because of the fact that Python needs many spaces </small>

In [4]:
ord("h") #gets the unocode integer value of a character

104

In [5]:
[ord(x) for x in "hello"] #gets the unicode integer values of all characters in a string\
'''
We can't just use this for tokenization as many characters map to the same integer value and also not stable
UTF-8 takes every code point and encodes it into 1-4 bytes. Each byte is an integer from 0-255
'''

"\nWe can't just use this for tokenization as many characters map to the same integer value and also not stable\nUTF-8 takes every code point and encodes it into 1-4 bytes. Each byte is an integer from 0-255\n"

In [6]:
list("hello".encode("utf-8"))  # encodes the string into bytes using utf-8 encoding
'''
the other utf encodings are utf-16 and utf-32 and are pretty wasteful
We want to be able to support larger vocabulary sizes so we use byte pair encoding (BPE)
BPE works by merging the most common pairs of bytes iteratively to form a vocabulary of tokens
This way we can represent common words or subwords with a single token, while still being able
to represent rare words with multiple tokens.
To want to just have raw bytes in there, you have to change the structure of the transformer,
theres a research paper on it: MEGABYTE: Predicting Million-byte sequences with multiscale transformers
'''

'\nthe other utf encodings are utf-16 and utf-32 and are pretty wasteful\nWe want to be able to support larger vocabulary sizes so we use byte pair encoding (BPE)\nBPE works by merging the most common pairs of bytes iteratively to form a vocabulary of tokens\nThis way we can represent common words or subwords with a single token, while still being able\nto represent rare words with multiple tokens.\nTo want to just have raw bytes in there, you have to change the structure of the transformer,\ntheres a research paper on it: MEGABYTE: Predicting Million-byte sequences with multiscale transformers\n'

In [7]:
text = "long sentence here: The quick brown fox jumps over the lazy dog."
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert bytes to integers
print('---')
print (text)
print("length: ", len(text))
print('---')
print(tokens)
print("length: ", len(tokens))

---
long sentence here: The quick brown fox jumps over the lazy dog.
length:  64
---
[108, 111, 110, 103, 32, 115, 101, 110, 116, 101, 110, 99, 101, 32, 104, 101, 114, 101, 58, 32, 84, 104, 101, 32, 113, 117, 105, 99, 107, 32, 98, 114, 111, 119, 110, 32, 102, 111, 120, 32, 106, 117, 109, 112, 115, 32, 111, 118, 101, 114, 32, 116, 104, 101, 32, 108, 97, 122, 121, 32, 100, 111, 103, 46]
length:  64


In [8]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): #Pythonic way to iterate over adjacent elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(stats)
print(sorted(((value, key) for key, value in stats.items()), reverse=True))  #print 10 most common byte pairs

'''
So this gives us the frequency of each adjacent byte pair in the token list
It wil be a dictionary where the keys are tuples of byte pairs and the values are their counts
We can use this to find the most common byte pairs and merge them iteratively to form our BPE vocabulary
'''


{(108, 111): 1, (111, 110): 1, (110, 103): 1, (103, 32): 1, (32, 115): 1, (115, 101): 1, (101, 110): 2, (110, 116): 1, (116, 101): 1, (110, 99): 1, (99, 101): 1, (101, 32): 3, (32, 104): 1, (104, 101): 3, (101, 114): 2, (114, 101): 1, (101, 58): 1, (58, 32): 1, (32, 84): 1, (84, 104): 1, (32, 113): 1, (113, 117): 1, (117, 105): 1, (105, 99): 1, (99, 107): 1, (107, 32): 1, (32, 98): 1, (98, 114): 1, (114, 111): 1, (111, 119): 1, (119, 110): 1, (110, 32): 1, (32, 102): 1, (102, 111): 1, (111, 120): 1, (120, 32): 1, (32, 106): 1, (106, 117): 1, (117, 109): 1, (109, 112): 1, (112, 115): 1, (115, 32): 1, (32, 111): 1, (111, 118): 1, (118, 101): 1, (114, 32): 1, (32, 116): 1, (116, 104): 1, (32, 108): 1, (108, 97): 1, (97, 122): 1, (122, 121): 1, (121, 32): 1, (32, 100): 1, (100, 111): 1, (111, 103): 1, (103, 46): 1}
[(3, (104, 101)), (3, (101, 32)), (2, (101, 114)), (2, (101, 110)), (1, (122, 121)), (1, (121, 32)), (1, (120, 32)), (1, (119, 110)), (1, (118, 101)), (1, (117, 109)), (1, (117,

'\nSo this gives us the frequency of each adjacent byte pair in the token list\nIt wil be a dictionary where the keys are tuples of byte pairs and the values are their counts\nWe can use this to find the most common byte pairs and merge them iteratively to form our BPE vocabulary\n'

In [10]:

top_pair = max(stats, key=stats.get) # get the byte pair with the highest count
top_pair

(101, 32)

In [15]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        #If we are not at the very last position AND the pair matches, replace it 
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)  #append the new merged token
            i += 2  #skip the next token as it's part of the merged pair
        else:
            newids.append(ids[i])  #append the current token as is
            i += 1
        return newids
print(merge([5,6,6,7,9,1], (6,7), 99)) #example usage
tokens = merge(tokens, top_pair, 256)  #merge the most common pair and assign it a new token id (256)
print(tokens)
print(len(tokens))

[5]
[108]
1


In [16]:
'''
Now do it again, iteratively, the more steps we take, the larger our vocabulary 
 becomes and shorter our token sequences
 ''' 
text = '''A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 25 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception. A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view. I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here. Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article. Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity. Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity! The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database. The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes. Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether. White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes. Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more. (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.) Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use. Scripts Let’s zoom in on the first three planes, since that’s where the action is: Map of scripts in Unicode planes 0–2 (click to zoom) This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram. Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used). Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white. You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1. Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file? The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text. Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string. Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower. UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value. UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works: UTF-8 (binary)	Code point (binary)	Range 0xxxxxxx	xxxxxxx	U+0000–U+007F 110xxxxx 10yyyyyy	xxxxxyyyyyy	U+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz	xxxxyyyyyyzzzzzz	U+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww	xxxyyyyyyzzzzzzwwwwww	U+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing. Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere. However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else. UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words. Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes: UTF-16 (binary)	xxxxxxxxxxxxxxxx	Code point (binary)	Range xxxxxxxxxxxxxxxx	U+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy	xxxxxxxxxxyyyyyyyyyy + 0x10000	U+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all. Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points. Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊) Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point! Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points. In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter. If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called combining marks, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character. For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text. Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢ṅ̨̺̝͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵̦o̰̥͓̣̫̙̤͢ṫ̞̞̲͉̲̻ ̳̪̠͖̳̯̕a̫͜n̼͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓ Further reading: The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points The Many Meanings of “Shader”Quadrilateral Interpolation, Part 2 25 Comments on “A Programmer’s Introduction to Unicode” Texture Gathers and Coordinate Precision git-partial-submodule Slope Space in BRDF Theory Hash Functions for GPU Rendering All Posts Categories Graphics(32) Coding(23) Math(21) GPU(15) Physics(6) Eye Candy(4) © 2007–2025 by Nathan Reed. Licensed CC-BY-4.0.'''
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert bytes to integers

In [18]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): #Pythonic way to iterate over adjacent elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        #If we are not at the very last position AND the pair matches, replace it 
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)  #append the new merged token
            i += 2  #skip the next token as it's part of the merged pair
        else:
            newids.append(ids[i])  #append the current token as is
            i += 1
    return newids
# ---
vocab_size = 276 #The desired final vocabulary size
num_merges = vocab_size - 256  #Initial byte vocab size is 256
ids = list(tokens)  #Make a copy of the original token list

merges = {} #(int, int) -> int
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get) # get the byte pair with the highest count
    idx = 256 + i  #New token index
    merges[pair] = idx
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)  #merge the most common pair and assign it a new token id
    merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (99, 111) into a new token 260
merging (116, 32) into a new token 261
merging (101, 114) into a new token 262
merging (97, 110) into a new token 263
merging (44, 32) into a new token 264
merging (111, 114) into a new token 265
merging (100, 32) into a new token 266
merging (226, 128) into a new token 267
merging (260, 100) into a new token 268
merging (97, 114) into a new token 269
merging (101, 110) into a new token 270
merging (121, 32) into a new token 271
merging (46, 32) into a new token 272
merging (257, 103) into a new token 273
merging (97, 108) into a new token 274
merging (259, 256) into a new token 275


In [20]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print("compression ratio:", f"{len(tokens)/len(ids):.2f}X")

tokens length: 15437
ids length: 12200
compression ratio: 1.27X


<small> Note: Tokenizer is a completely separate, independent module from the LLM. Has its own trainign set of text,
which can be different from the LLM, on which you train the vocabulary using the BPE. It then translates back 
and forth between raw text and sequences of tokens. The LLM later only ever sees the tokens and doesnt directly
deal with the text. </small>

In [34]:
#Now at decoding
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1] #good thing we are using modern Python, where the dict preserves insertion order

def decode(ids):
    #given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids) #b"" is one way to make a bytes object"
    text = tokens.decode("utf-8", errors="replace")  
    return text

print(decode([128])) #utf-8 cannot decode byte 128 on its own because theres a specific schema for multi-byte objects

�


In [40]:
def encode(text):
    #given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))  #find the first pair in merges
        '''
        Function might fail if there is nothing to merge anymore

        '''
        if pair not in merges:
            break #nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens
        
print(encode("hello world!"))



[104, 101, 108, 108, 111, 32, 119, 265, 108, 100, 33]


In [41]:
print(decode(encode("hello world!")))

hello world!


<small> Merged splits with regex patterns (GPT series) </small>

<small> So the issue is that you are allowing for different combinations of basically the same word and merging them together like dog, Dog, dog!. So this is considered suboptimal. For GPT-2, the used import regex as re, which is actually an extension of re. 

In [57]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?[0-9]+| ?[^\s\w]+|\s+(?!\S)|\s+""")
#The vertical barls are or's, you go left to right from this pattern and 
# try to match it against the string where you are

print(re.findall(gpt2pat, "Hello've world123 HOW'S are       you!!!?"))
'''
These are going to be conconated to form tokens in GPT-2 style tokenization
you are never going to be merging the across the boundaries of these tokens
this is how regex makes it so some merges cannot happen
gpt-2 did not use ignoreUppercase, so it brings issues when tokenizing
'''

['Hello', "'ve", ' world', '123', ' HOW', "'", 'S', ' are', '      ', ' you', '!!!?']


'\nThese are going to be conconated to form tokens in GPT-2 style tokenization\nyou are never going to be merging the across the boundaries of these tokens\nthis is how regex makes it so some merges cannot happen\ngpt-2 did not use ignoreUppercase, so it brings issues when tokenizing\n'

In [61]:
import tiktoken as tk
#GPT-2 (does not merge spaces)
enc = tk.get_encoding("gpt2")
print(enc.encode("   Hello world??!!?"))

#GPT-4 (merges spaces)
enc = tk.get_encoding("cl100k_base")
print(enc.encode("   Hello world??!!?"))

[220, 220, 18435, 995, 3548, 3228, 30]
[256, 22691, 1917, 7801, 3001, 30]
[256, 22691, 1917, 7801, 3001, 30]


In [68]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f)
with open('C:\\Users\\apurv\\Downloads\\Apurva downloads\\Python\\GPT-Tokenizer\\vocab.bpe', 'r', encoding='utf-8') as f:
    bpe_data = f.read()
bpe_merges = [tuple(line.split()) for line in bpe_data.split('\n')[1:-1]]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 936: character maps to <undefined>

special tokens

In [ ]:
len(encoder) #256 raw byte tokens. 50,000 merges. +1 special token

NameError: name 'encoder' is not defined